# 3.3. Manejo de overscan y bias para marcos oscuros 
# 3.3.1. Las opciones para reducir los marcos oscuros 
Los siguientes pasos a seguir dependen de dos cosas:

¿Estás restando overscan? Si es así, debe restar el overscan para los marcos oscuros.

¿Necesitará escalar estos oscuros a un tiempo de exposición diferente? Si es así, debe restar el sesgo de los oscuros. Si no, deja el sesgo.

# 3.3.1.1. 1. ¿Necesita restar overscan? 
Si decide restar la sobreexploración de cualquiera de las imágenes utilizadas en la reducción de datos, debe sustraer la sobreexploración de todas las imágenes. Esto incluye los oscuros y es independiente de si tiene la intención o no de escalar los fotogramas oscuros a otros tiempos de exposición.

Úselo ccdproc.subtract_overscanpara eliminar el overscan. Vea el cuaderno XX para una discusión sobre la sobreexploración, y vea YY para un ejemplo práctico en el que se resta la sobreexploración.

# 3.3.1.2. 2. ¿Necesita escalar los oscuros? 
Depende de los tiempos de exposición de las otras imágenes que necesites reducir. Si las otras imágenes tienen tiempos de exposición que coinciden con sus marcos oscuros, entonces no necesita escalar los oscuros. Si alguna otra imagen (excepto los marcos de sesgo, que siempre tienen exposición cero) tiene un tiempo de exposición que no coincide con los tiempos de exposición de los oscuros, entonces deberá escalar los oscuros por tiempo de exposición.

Si necesita escalar los marcos oscuros, debe restarles el sesgo usando ccdproc.subtract_bias. Los ejemplos de uso ccdproc.subtract_biasestán en el siguiente cuaderno.

Si no necesita escalar los cuadros oscuros a un tiempo de exposición diferente, no reste el sesgo de los oscuros. Los marcos oscuros te servirán para eliminar tanto el sesgo como la oscuridad actual de tus imágenes.

Como recordatorio, debe esforzarse mucho para evitar escalar los cuadros oscuros a un tiempo de exposición más largo porque principalmente aumentará el ruido en lugar de la corriente oscura.

# 3.4. Calibrar imágenes oscuras 
Las imágenes oscuras, como cualquier otra imagen, deben calibrarse. Según los datos que tenga y las opciones que haya tomado para reducir sus datos, los pasos para reducir sus imágenes pueden incluir:

Restar overscan (solo si decide restar overscan de todas las imágenes).

Recorte la imagen (si tiene overscan, ya sea que esté usando overscan o no).

Reste el sesgo (si necesita escalar los marcos oscuros calibrados a un tiempo de exposición diferente).

In [1]:
from pathlib import Path

from astropy.nddata import CCDData
from ccdproc import ImageFileCollection
import ccdproc as ccdp

# 3.4.1. Ejemplo 1: Overscan restado, sesgo no eliminado 
# 3.4.1.1. Fíjate qué imágenes tienes 
Primero recopilamos información sobre las imágenes en bruto y las imágenes reducidas hasta este punto. Estos ejemplos tienen oscuros almacenados en un subdirectorio de la carpeta con el resto de las imágenes, por lo que creamos un ImageFileCollectionpara cada uno.

In [2]:
ex1_path_raw = Path('example-cryo-LFC')

ex1_images_raw = ImageFileCollection(ex1_path_raw)
ex1_darks_raw = ImageFileCollection(ex1_path_raw / 'darks')

ex1_path_reduced = Path('example1-reduced')
ex1_images_reduced = ImageFileCollection(ex1_path_reduced)

# 3.4.1.1.1. Imágenes crudas, todo excepto los oscuros


In [3]:
ex1_images_raw.summary['file', 'imagetyp', 'exptime', 'filter']

file,imagetyp,exptime,filter
str14,str9,float64,str2
ccd.001.0.fits,BIAS,0.0,i'
ccd.002.0.fits,BIAS,0.0,i'
ccd.003.0.fits,BIAS,0.0,i'
ccd.004.0.fits,BIAS,0.0,i'
ccd.005.0.fits,BIAS,0.0,i'
ccd.006.0.fits,BIAS,0.0,i'
ccd.014.0.fits,FLATFIELD,70.001,g'
ccd.015.0.fits,FLATFIELD,70.011,g'
ccd.016.0.fits,FLATFIELD,70.001,g'


# 3.4.1.1.2. Marcos oscuros crudos

In [4]:
ex1_darks_raw.summary['file', 'imagetyp', 'exptime', 'filter']

file,imagetyp,exptime,filter
str14,str4,float64,str2
ccd.002.0.fits,BIAS,0.0,r'
ccd.013.0.fits,DARK,300.0,r'
ccd.014.0.fits,DARK,300.0,r'
ccd.015.0.fits,DARK,300.0,r'
ccd.017.0.fits,DARK,70.0,r'
ccd.018.0.fits,DARK,70.0,r'
ccd.019.0.fits,DARK,70.0,r'
ccd.023.0.fits,DARK,7.0,r'
ccd.024.0.fits,DARK,7.0,r'


# 3.4.1.2. Decidir qué pasos de calibración tomar 
Este ejemplo es, de nuevo, uno de los chips de la cámara LFC de Palomar. En portátiles anteriores, hemos visto que el chip tiene una región de sobreexploración útil , tiene poca corriente oscura excepto por algunos píxeles calientes y el sensor brilla en una esquina del chip.

Mirando la lista de imágenes no oscuras (es decir, las imágenes planas y claras) se muestra que para cada tiempo de exposición en las imágenes no oscuras hay un conjunto de exposiciones oscuras que tienen un tiempo de exposición coincidente o muy similar. .

Para ser más explícitos, hay flats con tiempos de exposición de 7,0 seg y 70,011 seg y darks con tiempo de exposición de 7,0 y 70,0 seg. Los tiempos de exposición oscuros y planos son lo suficientemente cercanos como para que no haya necesidad de escalarlos. Las dos imágenes de un objeto son cada una de aproximadamente 300 segundos, haciendo coincidir los oscuros con un tiempo de exposición de 300 segundos. La muy pequeña diferencia en el tiempo de exposición, inferior a 0,1 segundos, no necesita ser compensada.

Ante esto, vamos a:

Reste overscan de cada uno de los oscuros. La región de overscan útil es XXX (ver ENLACE).

Recorte el overscan de las imágenes oscuras.

No restaremos el sesgo de estas imágenes porque no necesitaremos volver a escalarlas a un tiempo de exposición diferente.

# 3.4.1.3. Calibrar los cuadros oscuros individuales 

In [5]:
for ccd, file_name in ex1_darks_raw.ccds(imagetyp='DARK',            # Just get the dark frames
                                         ccd_kwargs={'unit': 'adu'}, # CCDData requires a unit for the image if 
                                                                     # it is not in the header
                                         return_fname=True           # Provide the file name too.
                                        ):    
    # Subtract the overscan
    ccd = ccdp.subtract_overscan(ccd, overscan=ccd[:, 2055:], median=True)
    
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :2048])
    
    # Save the result
    ccd.write(ex1_path_reduced / file_name)

# 3.4.1.3.1. Imágenes reducidas (hasta ahora)

In [6]:
ex1_images_reduced.refresh()
ex1_images_reduced.summary['file', 'imagetyp', 'exptime', 'filter', 'combined']

file,imagetyp,exptime,filter,combined
str17,str4,float64,str2,object
ccd.001.0.fits,BIAS,0.0,i',--
ccd.002.0.fits,BIAS,0.0,i',--
ccd.003.0.fits,BIAS,0.0,i',--
ccd.004.0.fits,BIAS,0.0,i',--
ccd.005.0.fits,BIAS,0.0,i',--
ccd.006.0.fits,BIAS,0.0,i',--
ccd.013.0.fits,DARK,300.0,r',--
ccd.014.0.fits,DARK,300.0,r',--
ccd.015.0.fits,DARK,300.0,r',--


# 3.4.2. Ejemplo 2: Overscan no se resta, se elimina el sesgo

In [7]:
ex2_path_raw = Path('example-thermo-electric')

ex2_images_raw = ImageFileCollection(ex2_path_raw)

ex2_path_reduced = Path('example2-reduced')
ex2_images_reduced = ImageFileCollection(ex2_path_reduced)

In [8]:
ex2_images_raw.summary['file', 'imagetyp', 'exposure'].show_in_notebook()

idx,file,imagetyp,exposure
0,AutoFlat-PANoRot-r-Bin1-001.fit,FLAT,1.0
1,AutoFlat-PANoRot-r-Bin1-002.fit,FLAT,1.0
2,AutoFlat-PANoRot-r-Bin1-003.fit,FLAT,1.0
3,AutoFlat-PANoRot-r-Bin1-004.fit,FLAT,1.0
4,AutoFlat-PANoRot-r-Bin1-005.fit,FLAT,1.0
5,AutoFlat-PANoRot-r-Bin1-006.fit,FLAT,1.02
6,AutoFlat-PANoRot-r-Bin1-007.fit,FLAT,1.06
7,AutoFlat-PANoRot-r-Bin1-008.fit,FLAT,1.11
8,AutoFlat-PANoRot-r-Bin1-009.fit,FLAT,1.16
9,AutoFlat-PANoRot-r-Bin1-010.fit,FLAT,1.21


# 3.4.2.1. Decida qué pasos tomar a continuación 
En este caso los únicos fotogramas oscuros tienen un tiempo de exposición de 90 seg. Aunque eso coincide con el tiempo de exposición de las imágenes científicas, las imágenes de campo plano tienen un tiempo de exposición mucho más corto, que va de 1 segundo a 1,21 segundos. Este tipo de rango de exposición es típico cuando se toman planos crepusculares. Dado que estos son tiempos de exposición muy diferentes a los oscuros, será necesario escalar los marcos oscuros.

Recuerde que para esta cámara, el overscan no es útil y debe recortarse.

Ante esto, vamos a:

Recorte el overscan de cada uno de los marcos oscuros.

Reste el sesgo de calibración de los cuadros oscuros para que podamos escalar los oscuros a un tiempo de exposición diferente.

# 3.4.2.2. Calibración de los cuadros oscuros individuales 
Primero, leemos la imagen de polarización combinada creada en el cuaderno anterior. Aunque podríamos hacer esto en función del nombre del archivo, es menos probable que se produzcan errores utilizando un conjunto sistemático de palabras clave de encabezado para realizar un seguimiento de las imágenes que se han combinado.

In [9]:
combined_bias = CCDData.read(ex2_images_reduced.files_filtered(imagetyp='bias', 
                                                               combined=True, 
                                                               include_path=True)[0])

In [10]:
for ccd, file_name in ex2_images_raw.ccds(imagetyp='DARK',            # Just get the bias frames
                                          return_fname=True           # Provide the file name too.
                                         ):
        
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :4096])
    
    # Subtract bias
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    # Save the result
    ccd.write(ex2_path_reduced / file_name)